# Calculate sidewalk width
Using centerline extraction (also called skeleton line, axis line, or medial line extraction)

In [1]:
# Add project src to path.
import set_path

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from shapely.ops import linemerge
from shapely.geometry import box

import geopandas as gpd
from centerline.geometry import Centerline

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import upcp.utils.bgt_utils as bgt_utils
import upcp.utils.las_utils as las_utils

import upc_sw.poly_utils as poly_utils

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Read the sidewalk and obstacle data

In [2]:
# Read sidewalk with obstacle data
df = gpd.read_file('../datasets/obstacles/sidewalks_with_obstacles.shp')

# A CRS tells Python how those coordinates relate to places on the Earth. Rijksdriehoek = epsg:28992
crs = 'epsg:28992' #local crs

## Merge the sidewalk polygons

In [3]:
df_dissolved = gpd.GeoDataFrame(geometry=gpd.GeoSeries([geom for geom in df.unary_union.geoms]))
df_exploded = gpd.GeoDataFrame(df_dissolved.geometry.explode(index_parts=True))

In [4]:
# Ignore sidewalk polygons that are too small
min_area_size = 5
df_exploded = df_exploded[df_exploded.geometry.area > min_area_size]

## Calculate the centerlines

In [5]:
# if you get an error here, make sure you use tqdm>=4.61.2
df_exploded['centerlines'] = df_exploded.progress_apply(lambda row: Centerline(row.geometry), axis=1)
df_exploded = df_exploded.set_geometry('centerlines')

  0%|          | 0/6 [00:00<?, ?it/s]

## Remove Short Line Ends

In [6]:
df_exploded['centerlines'] = df_exploded['centerlines'].progress_apply(linemerge)

  0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
df_exploded['centerlines'] = df_exploded['centerlines'].progress_apply(poly_utils.remove_short_lines)

  0%|          | 0/6 [00:00<?, ?it/s]

## Get Sidewalk Widths

In [8]:
df_exploded['centerlines'] = df_exploded['centerlines'].progress_apply(lambda row: row.simplify(0.2, preserve_topology=True))

  0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
df_exploded['segments'] = df_exploded['centerlines'].progress_apply(poly_utils.get_segments)

  0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
df_exploded[['avg_distances', 'min_distances']] = df_exploded.progress_apply(lambda row: poly_utils.get_avg_distances(row), axis=1)

  0%|          | 0/6 [00:00<?, ?it/s]

In [11]:
data = {'geometry': [], 'avg_width': [], 'min_width': []}

for i, row in df_exploded.iterrows():
    
    for segment in row.segments:
        data['geometry'].append(segment)
    
    for avg_distance in row.avg_distances:
        data['avg_width'].append(avg_distance * 2)
    
    for min_distance in row.min_distances:
        data['min_width'].append(min_distance * 2)
        
df_segments = pd.DataFrame(data)
df_segments = gpd.GeoDataFrame(df_segments, crs=crs, geometry='geometry')

In [12]:
# Width in meters
df_segments['avg_width'] = round(df_segments['avg_width'] * 10) / 10
df_segments['min_width'] = round(df_segments['min_width'] * 10) / 10

## Check coverage of point cloud data on sidewalks

In [13]:
pc_data_folder = '../datasets/pointclouds/'

all_tiles = las_utils.get_tilecodes_from_folder(f'{pc_data_folder}m3c2/')

In [14]:
def get_tile_dimensions(tilecode):
    ((x_min, y_max), (x_max, y_min)) = las_utils.get_bbox_from_tile_code(tilecode, padding=0)
    tile_polygon = box(*[x_min, y_min, x_max, y_max], ccw=True)
    return tile_polygon

tile_dimensions = [get_tile_dimensions(tile) for tile in all_tiles]

In [15]:
df_tile_dimensions = gpd.GeoDataFrame(crs=crs, geometry=tile_dimensions)

# When True, take the union of the polygons
if len(tile_dimensions) > 1:
    df_tile_dimensions = gpd.GeoDataFrame(geometry=gpd.GeoSeries([geom for geom in df_tile_dimensions.unary_union.geoms]))
    df_tile_dimensions = gpd.GeoDataFrame(df_tile_dimensions.geometry.explode(index_parts=True))

In [16]:
# Check if line segment intersects with point cloud tile
df_segments['pc_coverage'] = df_segments.apply(lambda row: df_tile_dimensions.intersects(row.geometry).any(), axis=1)

## Store output

In [17]:
df_projected = df_segments.to_crs('EPSG:4326')
with open('../datasets/output/intermediate_output_segments.geojson', 'w') as f:
    f.write(df_projected.to_json())